In [1]:
from __init__ import *
import pickle as pkl
import time
import os
import json
import os
from pathlib import Path

VERSION_ID = "V1_2"
def read_json(file_name):
    with open(file_name, "r") as f:
        obj = json.load(f)
    return obj

def getDatasetsList(logdir):
    filename = Path(logdir) / 'datasetlist_{}.json'.format(VERSION_ID)
    if not os.path.isfile(filename):
        print("{} is not existed".format(filename))
        return None
    else:
        datasets_released_list = read_json(filename)
        return datasets_released_list

def getGraphBundleInfo(logdir, dataset_id):
    filename = Path(logdir) / 'cache_bundle_{}_{}.json'.format(dataset_id, VERSION_ID)
    if not os.path.isfile(filename):
        print("{} is not existed".format(filename))
        return None
    else:
        graph_pkg = read_json(filename)
        return graph_pkg    



In [2]:
graphbundle_obj = getGraphBundleInfo("/data2/zhihua/github/GNNLens/gnnlens/public/data/", 4)

In [3]:
graphbundle_obj.keys()

dict_keys(['graph_obj', 'success'])

In [4]:
graphbundle_obj["graph_obj"].keys()

dict_keys(['common', 'individual'])

In [15]:
graphbundle_obj["graph_obj"]["individual"]["GCN"]["model"]

4

In [ ]:
'graph_out', 'message_passing', 'model', 'real_model_name'

In [ ]:
dict_keys(['node_features', 'output_vector'])


In [ ]:
dict_keys(['GCN', 'GCN_Identity_features', 'MLP'])


In [7]:
graphbundle_obj["graph_obj"]["common"]["mask"].keys()

dict_keys(['test', 'train', 'valid'])

In [ ]:
def getGraphBundleInfo(dataset_id):
    if dataset_id in dataset_keys:
        graph_bundle_info = graphs_list["graphs"][dataset_id]["bundle_info"]
        explain_id = 4
        graph_id = 1
        return_package = {}
        common_flag = False
        common_package = {}
        individual_package = {}
        for key in graph_bundle_info:
            model_id = graph_bundle_info[key]["model_id"]
            real_model_name = graph_bundle_info[key].get("real_model_name", DefaultNameMapping[key])
            graph_info = getGraphInfo(dataset_id, model_id, explain_id, graph_id)
            return_package[key] = graph_info
            if not common_flag:
                '''
                common_package = {
                    "dataset_id": graph_info["dataset_id"],
                    "data_type_id": graph_info["data_type_id"],
                    "graph": graph_info["graph"],
                    "explain_id": graph_info["explain_id"],
                    "task": graph_info["task"],
                    "graph_in": graph_info["graph_in"],
                    "graph_target":graph_info["graph_target"],
                    "graph_explaination":graph_info["graph_explaination"],
                    "graph_layout":graph_info["graph_layout"], 
                    "embedding":graph_info["embedding"],
                    "mask":graph_info["mask"],
                    "graph_additional_info":graph_info["graph_additional_info"],
                    "name":graph_info["name"]
                }'''
                common_package = {
                    "dataset_id": graph_info["dataset_id"],
                    "data_type_id": graph_info["data_type_id"],
                    "graph": graph_info["graph"],
                    "explain_id": graph_info["explain_id"],
                    "task": graph_info["task"],
                    "graph_in": graph_info["graph_in"],
                    "graph_target":graph_info["graph_target"],
                    "graph_explaination":graph_info["graph_explaination"],
                    "graph_layout":graph_info["graph_layout"], 
                    "mask":graph_info["mask"],
                    "graph_additional_info":graph_info["graph_additional_info"],
                    "name":graph_info["name"]
                }
                common_flag = True
            else:
                pass
            '''
            local_individual_package = {
                "model": graph_info["model"],
                "graph_out":graph_info["graph_out"],
                "message_passing":graph_info["message_passing"],
                "model_state_dict":graph_info["model_state_dict"],
            }
            '''
            
            local_individual_package = {
                "model": graph_info["model"],
                "graph_out":graph_info["graph_out"],
                "real_model_name": real_model_name
            }
            if key == "GCN":
                local_individual_package["message_passing"] = graph_info["message_passing"]
            
            individual_package[key] = local_individual_package
        return_package = {
            "common": common_package,
            "individual": individual_package
        }
        #if ENABLE_COMPRESSED:
        #    return_package = compress_data(return_package)
        if ENABLE_CACHE:
            saveCacheGraphBundleInfo(dataset_id, return_package)
        return return_package
        
    else:
        print("Not found dataset id {}".format(dataset_id))
        return None

In [ ]:
"""Dump data for GNNVis"""
import dgl.backend as F
import json
import numpy as np
import os
from dgl import DGLError

class GNNLensWriter():
    """
    Parameters
    ----------
    logdir: str
        Path to create a new directory for dumping data files, which can 
        be either a relative path or an absolute path.
    """
    def __init__(self, logdir, exist_ok=False):
        os.makedirs(logdir, exist_ok=exist_ok)
        self.logdir = logdir
        self.graph_names = []
        self.graph_data = dict()
        self.VERSION_ID = "V1_2"
        
    def _get_graph_logdir(self, name):
        """Get logdir for the graph.
        
        Parameters
        ----------
        name : str
            Name of the graph.
        """
        return os.path.join(self.logdir, str(self.graph_data[name]['dataset_id']))
    
    def add_graph(self, name, graph, nlabels, num_nlabel_types, features, mask, graph_additional_info):
        """Add data for a graph.

        Parameters
        ----------
        name : str
            Name of the graph.
        graph : DGLGraph
            A homogeneous graph.
        nlabels : Tensor of integers, optional
            Node labels. The tensor can be reshaped as (N,) where N is the number of nodes.
            Each node should be associated with one label only.
        num_nlabel_types : int, optional
            Number of node label types. If not provided and nlabels is provided,
            this will be inferred then. num_nlabel_types should be no greater than 10.
        eweights : dict[str, tensor]
            Edge weights. The keys are the eweight names, e.g. confidence. The values are the
            tensors of edge weights. The tensors can be reshaped as (E,) where E is the number
            of edges.
        """
        if name in self.graph_names:
            raise ValueError('Graph name {} has already been used.'.format(name))

        num_nodes = graph.num_nodes()
        num_edges = graph.num_edges()
        
        srcs, dsts = graph.edges()
        srcs = F.asnumpy(srcs).tolist()
        dsts = F.asnumpy(dsts).tolist()

        # Handle nlabels
        if nlabels is None:
            nlabels = []
        else:
            nlabels = F.asnumpy(nlabels)
            try:
                nlabels = np.reshape(nlabels, (num_nodes,))
            except:
                raise DGLError('Node labels should be able to be reshaped as (num_nodes,)')
            if num_nlabel_types is None:
                num_nlabel_types = int(nlabels.max()) + 1
            assert num_nlabel_types <= 10, \
                'Expect num_nlabel_types to be no greater than 10, got {:d}'.format(
                    num_nlabel_types)
            nlabels = nlabels.tolist()
        dataset_id = graph_id
        data_type = 2
        graph_id = 1
        explain_id = 4
        task = "node_classification"
        
        ## graph_in
        features_sparse, features_sparse_value = constructSparseFeature(features.cpu().numpy())
        original_graph = whole_graph["graph"]
        original_graph = whole_graph["graph"]
        Explain_type = "MessagePassing"
        graph_layout = []
        common_package = {
                "dataset_id": dataset_id,
                "data_type_id": data_type,
                "graph": graph_id,
                "explain_id": explain_id,
                "task": task,
                "graph_in": {
                    "feature":features_sparse,
                    "feature_value":features_sparse_value,
                    "senders":original_graph._indices()[1].tolist(),
                    "receivers":original_graph._indices()[0].tolist()
                },
                "graph_target":{
                    "node_features":nlabels,
                },
                "graph_explaination":
                {
                    "type":  Explain_type
                },
                "graph_layout":graph_layout, 
                "mask":mask,
                "graph_additional_info":graph_additional_info,
                "name":name
        }
        '''
        local_individual_package = {
            "model": graph_info["model"],
            "graph_out":graph_info["graph_out"],
            "real_model_name": real_model_name
        }

        individual_package[key] = local_individual_package
        '''
        individual_package = {}
        return_package = {
            "common": common_package,
            "individual": individual_package
        }
        # Register graph name
        self.graph_names.append(name)
        self.graph_data[name] = return_package

    def add_model(self, graph_name, model_name, nlabels, eweights=None):
        """Add data for a model
        
        Parameters
        ----------
        graph_name : str
            Name of the graph.
        model_name : str
            Nmae of the model.
        nlabels : Tensor of integers, optional
            Node labels. The tensor can be reshaped as (N,) where N is the number of nodes.
            Each node should be associated with one label only.
        eweights : dict[str, tensor]
            Edge weights. The keys are the eweight names, e.g. confidence. The values are the
            tensors of edge weights. The tensors can be reshaped as (E,) where E is the number
            of edges. The edge weights should be in range [0, 1].
        """
        assert graph_name in self.graph_names, \
            'Expect add_graph to be called first for graph {}'.format(graph_name)

        
        # Handle nlabels
        nlabels = F.asnumpy(nlabels)
        num_nodes = self.graph_data[graph_name]['num_nodes']
        try:
            nlabels = np.reshape(nlabels, (num_nodes,))
        except:
            raise DGLError('Node labels should be able to be reshaped as (num_nodes,)')
        nlabels = nlabels.tolist()

        
        # Dump model data file
        num_models = len(self.graph_data[graph_name]['model_list']) + 1
        model_obj = {
            "name": model_name,
            "nlabels": nlabels,
            "eweights": eweights
        }
        with open(graph_logdir + '/model_{}.json'.format(num_models), 'w') as f:
            json.dump({"model_obj": model_obj, "success": True}, f)

        # Register the model
        self.graph_data[graph_name]['model_list'].append(model_name)

    def flush(self):
        """Finish dumping data."""
        VERSION_ID = self.VERSION_ID
        # Dump data list (meta info)
        with open(self.logdir + '/datasetlist_{}.json'.format(VERSION_ID), 'w') as f:
            datasets = []
            for i, name in enumerate(self.graph_names):
                datasets.append({"id": i + 1, "name": name})
            json.dump({"datasets": datasets, "success": True}, f)
        
        for name in self.graph_names:
            graph_id = self.graph_data[name]['id']
            VERSION_ID = self.VERSION_ID
            # Dump model meta info
            with open(self.logdir + '/cache_bundle_{}_{}.json'.format(graph_id, VERSION_ID), 'w') as f:
                json.dump({"success": True}, f)
        
